# Emotion Tracking Agent 테스트 (Production Level)

감정 추적 에이전트 (Level 1) 테스트 노트북

## 역할: "Emotional Intelligence Analyst" (감정 분석가)
- 캐릭터별 감정 상태 추적
- 감정 강도 및 트리거 분석
- 숨겨진 감정 탐지

> **Note**: 캐릭터 이름은 Character Agent의 이름과 정확히 매칭되어야 합니다.

In [ ]:
import sys, os, json, asyncio
project_root = os.path.abspath(os.path.join(os.getcwd(), '..', '..'))
if project_root not in sys.path: sys.path.insert(0, project_root)
from dotenv import load_dotenv
load_dotenv(os.path.join(project_root, '.env'))
print(f"Project root: {project_root}")

In [ ]:
SAMPLE_STORY = """
서진은 분노로 가득 차 있었다. 한때 친구였던 이민호가 마을을 배신했다는 사실이 믿기지 않았다.

하나는 걱정스러운 표정으로 두 사람을 바라보았다. 그녀는 이 상황이 폭력으로 끝나지 않기를 바랐다.

이민호의 눈에는 슬픔과 분노가 섞여 있었다. 배신이라고? 그들이 먼저 배신했다. 
그의 마음 속 깊은 곳에는 복수심이 불타고 있었지만, 동시에 옛 친구를 마주하는 것이 고통스러웠다.

박서연은 두려움에 떨며 나타났다. "경비병들이 온다!" 그녀의 목소리에는 공포가 묻어났다.

서진은 잠시 망설였다. 분노와 슬픔, 그리고 옛 우정에 대한 그리움이 뒤섞였다.
"""

EXTRACTED_CHARACTERS = [
    {"name": "서진", "role": "주인공"},
    {"name": "이민호", "role": "적대자"},
    {"name": "하나", "role": "조력자"},
    {"name": "박서연", "role": "조력자"}
]

def create_base_state(story=SAMPLE_STORY):
    return {"content": story, "project_id": "test-001", "document_id": "doc-001", 
            "job_id": "job-001", "callback_url": "http://localhost:8080/internal/callback",
            "extracted_characters": EXTRACTED_CHARACTERS,
            "tracked_emotions": {}, "messages": [], "errors": []}

def run_async(coro):
    try:
        loop = asyncio.get_event_loop()
        if loop.is_running():
            import nest_asyncio; nest_asyncio.apply()
            return loop.run_until_complete(coro)
        return asyncio.run(coro)
    except: return asyncio.run(coro)

## 1. Emotion Tracking 실행

In [ ]:
from app.agents.extraction.emotion import emotion_tracking_node

async def test_emotion_tracking():
    print("💖 Emotion Tracking (Production Level) 테스트...")
    state = create_base_state()
    return await emotion_tracking_node(state)

result = run_async(test_emotion_tracking())
emotions = result.get("tracked_emotions", {})

print(f"\n✅ 분석 완료:")
print(f"   - emotion_states: {len(emotions.get('emotion_states', []))}개")
print(f"   - neo4j_updates: {len(emotions.get('neo4j_updates', []))}개 (Machine-Readable)")

## 2. Emotion States (Human-Readable)

In [ ]:
emotion_states = emotions.get("emotion_states", [])

print("="*70)
print("💖 Emotion States (캐릭터별 감정 상태)")
print("="*70)

for em in emotion_states:
    char = em.get('character', 'N/A')
    primary = em.get('primary_emotion', 'N/A')
    secondary = em.get('secondary_emotion', '')
    intensity = em.get('intensity', 0)
    trigger = em.get('trigger', '')
    
    emotion_bar = '█' * intensity + '░' * (10 - intensity)
    
    print(f"\n🧑 {char}")
    print(f"   감정: {primary}" + (f" / {secondary}" if secondary else ""))
    print(f"   강도: [{emotion_bar}] {intensity}/10")
    if trigger:
        print(f"   트리거: {trigger}")
    if em.get('is_hidden'):
        print(f"   🎭 (숨겨진 감정)")

## 3. Neo4j-Ready JSON (Machine-Readable)

In [ ]:
neo4j_updates = emotions.get("neo4j_updates", [])

print("="*70)
print("🔗 Neo4j-Ready JSON (Character 노드 속성 업데이트용)")
print("="*70)
print(json.dumps(neo4j_updates, ensure_ascii=False, indent=2))
print(f"\n📊 Total updates: {len(neo4j_updates)}")

## 4. 참조 무결성 검사

In [ ]:
valid_names = {c["name"] for c in EXTRACTED_CHARACTERS}

print("="*70)
print("🔍 Referential Integrity Check")
print("="*70)
print(f"Valid names from Character Agent: {valid_names}")

integrity_errors = []
for em in emotion_states:
    char = em.get('character')
    if char and char not in valid_names:
        integrity_errors.append(f"Character '{char}' not in Character list")

if integrity_errors:
    print(f"\n❌ INTEGRITY ERRORS ({len(integrity_errors)}):")
    for err in integrity_errors:
        print(f"   - {err}")
else:
    print("\n✅ All names match Character Agent output - Referential Integrity OK!")

## 5. Spring Boot Callback 데이터

In [ ]:
callback_data = {
    "emotion_tracking": {
        "emotion_states": emotion_states,
        "neo4j_updates": neo4j_updates
    }
}

print("="*70)
print("📤 Spring Boot Callback 데이터 구조")
print("="*70)
print(json.dumps(callback_data, ensure_ascii=False, indent=2))

---

## 6. 💡 향후 개선: Event Sourcing 패턴

### ⚠️ 현재 `neo4j_updates`가 덮어쓰기(Overwrite)인 이유

#### 근거 1: 스칼라 값(Scalar Value) 매핑 구조
```json
"property_updates": {
  "current_emotion": "분노",       // String (Scalar)
  "emotion_intensity": 8,          // Integer (Scalar)
  "emotion_valence": "negative"    // String (Scalar)
}
```
- `current_emotion` Key에 할당된 Value가 **리스트(List/Array)**가 아닌 **단일 문자열(String)**
- 이전 값이 무엇이었든 새로운 값 "분노"가 그 자리를 **대체(Replace)**
- 이력이 남는 구조라면: `emotions: [{"time": 1, "val": "기쁨"}, {"time": 2, "val": "분노"}]` (Append Only)

#### 근거 2: CRUD 관점 - UPDATE vs INSERT
```cypher
// 현재 로직 (Property Update) - Destructive Operation
MATCH (c:Character) SET c.current_emotion = '분노'
```
- 기존 속성값을 **소멸**시키고 새 값으로 변경
- 이력 저장하려면 새로운 관계/노드를 **CREATE**해야 함

#### 근거 3: 섹션 7의 Event Sourcing과 비교
| 구분 | 현재 (`neo4j_updates`) | 고도화 (`event_graph_data`) |
|------|------------------------|-----------------------------|
| 연산 | SET (UPDATE) | CREATE (INSERT) |
| 대상 | 노드 속성 | 새 엣지/노드 |
| 이력 | ❌ 소실 | ✅ 보존 |

---

### 현재 방식 (State Update)
캐릭터 노드의 속성을 **덮어쓰기(Overwrite)** 합니다.  
→ '현재 상태' 조회에 효율적이나, **감정 변화의 역사(History)**는 소실됩니다.

```cypher
// 현재 방식: Character 노드 속성 업데이트
MATCH (c:Character {name: "서진"})
SET c.current_emotion = "분노",
    c.emotion_intensity = 8,
    c.emotion_valence = "negative"
```

### 고도화 방식 (Event Graph)
감정을 **별도 노드**로 만들고 **엣지로 연결**합니다.  
→ 스토리 진행에 따른 **감정 변화 궤적(Trajectory)** 분석 가능!

```cypher
// 고도화 방식: Emotion 노드 생성 + 관계 연결
MATCH (c:Character {name: "서진"})
CREATE (e:Emotion {
    type: "분노",
    intensity: 8,
    trigger: "이민호의 배신"
})
CREATE (c)-[:FELT {timestamp: datetime(), chapter: 3}]->(e)
```

### 비교

| 방식 | 장점 | 단점 |
|------|------|------|
| **State Update** | 단순, 빠름 | 히스토리 소실 |
| **Event Graph** | 감정 변화 추적 가능 | 복잡성 증가 |

## 7. Event Sourcing 방식 예시 (향후 구현 시 참고)

In [ ]:
# Event Sourcing 방식 데이터 예시 (향후 구현 시 참고)
event_graph_data = []

for em in emotion_states:
    event = {
        "character_name": em.get("character"),
        "emotion_node": {
            "type": em.get("primary_emotion"),
            "intensity": em.get("intensity"),
            "trigger": em.get("trigger", ""),
            "is_hidden": em.get("is_hidden", False)
        },
        "relationship": {
            "type": "FELT",
            "properties": {
                "chapter": 3,  # 챕터 정보 (실제로는 state에서 가져옴)
                "document_id": "doc-001"
            }
        }
    }
    event_graph_data.append(event)

print("="*70)
print("📈 Event Sourcing 방식 데이터 (INSERT/CREATE 연산 - 이력 보존)")
print("="*70)
print(json.dumps(event_graph_data, ensure_ascii=False, indent=2))

---

## 🏁 결론

**승인(Approve)**합니다. 참조 무결성 검사 로직이 포함되어 있다는 전제하에,  
이 결과물은 실제 DB 파이프라인에 투입 가능한 수준(**Production Ready**)입니다.

이제 이 두 에이전트(Dialogue, Emotion)를 하나의 파이프라인으로 연결하여,  
텍스트가 입력될 때마다 그래프가 동적으로 업데이트되는 루프를 구현하시면 됩니다.